In [1]:
from pathlib import Path
from util import repo_root
from omnibelt.jester import view, select, batchify, AutoFileJester
from verb.definitions import Definition, load_patterns
from verb.imports import *

import pandas as pd

In [2]:
datadir = repo_root() / 'data' / 'old'
paths = list(datadir.glob('*.csv'))
len(paths)

10

In [3]:
datadir

WindowsPath('C:/Users/anwan/OneDrive/Khan/research/alphageometry/data/old')

In [ ]:
# AutoFileJester(datadir, 'csv')

In [4]:
rawdefs = Path('../defs.txt').read_text()
symbs = {}
for block in rawdefs.split('\n\n'):
	lines = block.split('\n')
	if len(lines) == 1:
		symbs[lines[0].split()[0]] = len(lines[0].split()) - 1
	else:
		symbs[lines[0].split()[0]] = len(lines[1].split(','))
del symbs['=']
len(symbs)

68

In [5]:
symbs

{'angle_bisector': 1,
 'angle_mirror': 1,
 'circle': 1,
 'circumcenter': 1,
 'eq_quadrangle': 1,
 'eq_trapezoid': 1,
 'eq_triangle': 1,
 'eqangle2': 1,
 'eqdia_quadrangle': 1,
 'eqdistance': 1,
 'foot': 1,
 'free': 1,
 'incenter': 1,
 'incenter2': 4,
 'excenter': 1,
 'excenter2': 4,
 'centroid': 4,
 'ninepoints': 4,
 'intersection_cc': 1,
 'intersection_lc': 1,
 'intersection_ll': 1,
 'intersection_lp': 1,
 'intersection_lt': 1,
 'intersection_pp': 1,
 'intersection_tt': 1,
 'iso_triangle': 1,
 'lc_tangent': 1,
 'midpoint': 1,
 'mirror': 1,
 'nsquare': 1,
 'on_aline': 1,
 'on_aline2': 1,
 'on_bline': 1,
 'on_circle': 1,
 'on_line': 1,
 'on_pline': 1,
 'on_tline': 1,
 'orthocenter': 1,
 'parallelogram': 1,
 'pentagon': 5,
 'psquare': 1,
 'quadrangle': 4,
 'r_trapezoid': 1,
 'r_triangle': 1,
 'rectangle': 2,
 'reflect': 1,
 'risos': 1,
 's_angle': 1,
 'segment': 2,
 'shift': 1,
 'square': 2,
 'isquare': 2,
 'trapezoid': 1,
 'triangle': 3,
 'triangle12': 1,
 '2l1c': 4,
 'e5128': 2,
 '3peq

In [82]:
formals = [prob for path in paths for prob in pd.read_csv(path)['fl_statement'].tolist()]
len(formals)

1708

In [86]:
statements = [clause for formal in formals for line in formal.split(';') for clause in line.split(', ')]
len(statements)

6643

In [83]:
from verb.definitions import ClauseGenerator
from verb.verbalize import IndependentStatementVerbalization
import json, yaml, random
from tabulate import tabulate
from tqdm.notebook import tqdm
from collections import Counter

In [120]:
# ctx = Controller(ClauseGenerator())
# ctx.clear_cache()
# # ctx['formal'] = random.choice(statements)
# print(ctx['formal'])
# ctx['clause']
translator = IndependentStatementVerbalization(None)
def render(info):
	def render_clause(clause):
		pre = ' '.join(clause['let']) if clause.get('let') else ''
		args = ' '.join(clause['args'])
		if pre:
			return f'{pre} = {clause["symbol"]} {args}'
		return f'{clause["symbol"]} {args}'
	if isinstance(info, dict):
		return render_clause(info)
	if isinstance(info, list) and isinstance(info[0], dict):
		return ', '.join(render_clause(clause) for clause in info)
	return '; '.join(', '.join(render_clause(clause) for clause in line) for line in info)

def verb(formal):
	return translator.fl_2_nl(formal)

def parse(formal):
	def parse_clause(clause):
		if ' = ' in clause:
			pre, fn = clause.split(' = ')
		else:
			pre, fn = '', clause
		let = pre.split()
		sym, *args = fn.split()
		return {'let': let, 'symbol': sym, 'args': args}
	return [[parse_clause(clause) for clause in line.split(', ')] for line in formal.split(';')]

def disp(formal):
	return yaml.dump([[{k:v for k,v in clause.items() if v} for clause in clauses] for clauses in parse(formal)], default_flow_style=None)[:-1]
# parse(formals[0])

In [112]:
formal = random.choice(formals)
assert formal == render(parse(formal))
formal

'A B C = ieq_triangle A B C; D E F G = excenter2 D E F G C A B; H = intersection_tt H D G E B A F; I = lc_tangent I A F'

In [87]:
full = []
for fl in tqdm(statements):
	fl = fl.strip()
	info = parse(fl)[0][0]
	nl = translator.fl_2_nl(fl)
	full.append({'fl': fl, 'info': info, 'nl': nl})
# fl = random.choice(statements).strip()
# info = parse(fl)
# nl = translator.fl_2_nl(fl)
# print(fl)
# print(nl)
len(full)

  0%|          | 0/6643 [00:00<?, ?it/s]

6643

In [88]:
cnt = Counter({s: 0 for s in symbs})
cnt.update([x['info']['symbol'] for x in full])
print(tabulate(cnt.most_common(), headers=['Symbol', 'Count']))

Symbol              Count
----------------  -------
lc_tangent            212
eqdistance            204
on_bline              199
on_pline              199
on_circle             198
angle_bisector        196
on_opline             195
pentagon              194
triangle12            191
on_dia                189
iso_triangle          187
on_line               187
eq_trapezoid          182
r_triangle            182
risos                 178
trapezoid             169
angle_mirror          167
on_tline              163
eqdia_quadrangle      158
quadrangle            157
ieq_triangle          157
eq_quadrangle         154
rectangle             153
r_trapezoid           152
triangle              152
free                  150
isquare               149
segment               144
on_aline              132
eqangle3              112
psquare                65
midpoint               62
incenter               58
excenter2              56
intersection_cc        56
reflect                55
eqangle2    

In [114]:
todo = set(symbs)
trainset = []
evalset = []
for fl in tqdm(formals):
	info = parse(fl)
	clauses = [c for line in info for c in line]
	for clause in clauses:
		if clause['symbol'] in todo:
			trainset.append(info)
			for clause in clauses:
				todo.discard(clause['symbol'])
			break
	else:
		evalset.append(info)
len(trainset), len(evalset)

  0%|          | 0/1708 [00:00<?, ?it/s]

(41, 1667)

In [124]:
examples = {k: [] for k in symbs}
for info in trainset:
	clauses = [c for line in info for c in line]
	for clause in clauses:
		examples.setdefault(clause['symbol'], []).append(clause)
sum(len(v) for v in examples.values())

182

In [125]:
lines = [f'{symbol} ({len(examples[symbol][0]["args"])}):\n' + '\n'.join(f'  - {verb(render(ex))}' for ex in examples[symbol][:3]) for symbol in examples if len(examples[symbol]) > 0]
print('\n'.join(lines))

angle_bisector (4):
  - F bisects ∠DBF, ∠FBE.
  - H is on the angle bisector of ∠ADH & ∠HDG.
  - Point I is defined such that I is the bisector such that ∠ICG = ∠BCI.
angle_mirror (4):
  - Define point E such that line CD is the bisector of ∠ACE.
  - D is defined such that line BC is the bisector of ∠ABD.
  - Line ID is the bisector of ∠LIO.
circle (4):
  - E is defined such that E is the center of the circle passing through C, A, & D.
circumcenter (4):
  - D is a points such that D is the circumcenter of B, C, & A.
eq_quadrangle (4):
  - Define points A, C, D, and B such that A, D, C, and B forms a quadrilateral where line AD = line BC.
  - Let A, D, C, and B be points such that B, A, C, D forms a quadrilateral such that line BC is equal to line AD.
  - Let G, J, I, & H be points such that H, G, J, I forms a quadrilateral such that line HI = line GJ.
eq_trapezoid (4):
  - B, C, D, & A are point such that B, C, A, and D forms a trapezoid with line AD = line BC.
  - Define points B, C, 

In [164]:
# raw = Path('autoformalize.txt').read_text()
# print(txt)
# desc = yaml.safe_load(raw)
# print(yaml.dump({f'{s} ({len(examples[s][0]["args"])})': d for s, d in desc.items() if len(examples[s]) > 0}, default_flow_style=False))
# print(yaml.dump({f'{s} ({len(examples[s][0]["args"])})': verb(render(es[0])) for s, es in examples.items() if len(examples[s]) > 0}, default_flow_style=False))
# print(yaml.dump({f'{s} ({len(examples[s][0]["args"])})': verb(render(es[0])) + '\n' + disp(render(es[0])) for s, es in examples.items() if len(examples[s]) > 0}, default_flow_style=False))
print('\n'.join(f'{s} ({len(examples[s][0]["args"])}): {verb(render(es[0]))}\n{disp(render(es[0]))}' for s, es in examples.items() if len(examples[s]) > 0))

angle_bisector (4): F is a angle bisector such that ∠DBF = ∠FBE.
- - args: [F, D, B, E]
    symbol: angle_bisector
angle_mirror (4): Point E is defined such that line CD is the bisector of ∠ACE.
- - args: [E, A, C, D]
    let: [E]
    symbol: angle_mirror
circle (4): E is defined such that E is the center of circle EDCA.
- - args: [E, D, C, A]
    let: [E]
    symbol: circle
circumcenter (4): Let D be a point such that D is the circumcenter of C, A, B.
- - args: [D, B, C, A]
    let: [D]
    symbol: circumcenter
eq_quadrangle (4): B, C, A, and D forms a quadrilateral such that line AD = line BC.
- - args: [A, B, C, D]
    let: [A, B, C, D]
    symbol: eq_quadrangle
eq_trapezoid (4): A, D, C, & B forms a trapezoid with line BC = line AD.
- - args: [A, B, C, D]
    let: [A, B, C, D]
    symbol: eq_trapezoid
eq_triangle (3): Triangle FAD is an equilateral.
- - args: [F, A, D]
    let: [F]
    symbol: eq_triangle
eqangle2 (4): E is defined such that E is a point where ∠CAE is congruent to 

[[{'let': ['A', 'B', 'C', 'D'],
   'symbol': 'eq_trapezoid',
   'args': ['A', 'B', 'C', 'D']}],
 [{'let': ['E'], 'symbol': 'on_circle', 'args': ['E', 'D', 'B']}]]

In [138]:
print(yaml.dump(parse(formals[0]), default_flow_style=None))

- - args: [A, B, C, D]
    let: [A, B, C, D]
    symbol: eq_trapezoid
- - args: [E, D, B]
    let: [E]
    symbol: on_circle



In [22]:
print(formals)

['A B C D = eq_trapezoid A B C D; E = on_circle E D B', 'A B C D = r_trapezoid A B C D', 'A B C D = eq_quadrangle A B C D; E = on_circle E D A; F = eqdistance F E D C', 'A B C D = r_trapezoid A B C D; E = angle_mirror E A C D; F G H I = excenter2 F G H I D A B; J = excenter J A I B; K = intersection_pp K B H D F A G', 'A B C D = trapezoid A B C D; E F G H I = pentagon E F G H I; J = reflect J G E B; K = foot K F A E; L M N O = ninepoints L M N O D H F; P = on_tline P K F J', 'A B C D = isquare A B C D; E = midpoint E A C', 'A B C = risos A B C; D = angle_mirror D A B C', 'A B C = triangle12 A B C; D = lc_tangent D B C; E F G = ieq_triangle E F G', 'A B C D = r_trapezoid A B C D; E = on_line E D C', 'A B C D = eq_trapezoid A B C D; E = on_pline E A C B', 'A B C = triangle A B C; D = circumcenter D B C A; E = foot E C A D; F = on_bline F A B; G = lc_tangent G F A', 'A B C D = quadrangle A B C D; E = foot E A C B; F = eqdistance F E B A; G = on_dia G D A, eqangle3 G F A B D C', 'A B C D =

In [139]:
clauses = [f for f in formals if ',' in f]
len(clauses)

342

In [151]:
frm = random.choice(formals)
# frm = clauses[0]
# print(parse(frm))
print(translator.problem_fl_2_nl(frm))
print(disp(frm))

Define points C, A, and B such that C, A, & B is a right angle isosceles triangle with the right angle at A. Points D, G, F, and E are defined such that G & F are constructed such that DEFG forms a square.
- - args: [A, B, C]
    let: [A, B, C]
    symbol: risos
- - args: [D, E, F, G]
    let: [D, E, F, G]
    symbol: isquare


In [170]:
prob = random.choice(trainset)
prob = evalset[6]
print(translator.problem_fl_2_nl(render(prob)))
print(disp(render(prob)))

Let A, C, B be points such that ABC is a triangle ABC, such that A and B is half in length to that of B, C. Point D is defined such that D is a point where line DC = line AB.
- - args: [A, B, C]
    let: [A, B, C]
    symbol: triangle12
- - args: [D, C, A, B]
    let: [D]
    symbol: eqdistance


In [ ]:
print()

In [38]:
import re
from verb.definitions import ClauseGenerator

In [40]:
base_path = repo_root() / 'defs.txt'

unique_function_names = list(re.findall(r'\n\n\s*(\w+)', base_path.read_text(), re.MULTILINE))

path = repo_root() / 'assets' / 'def-patterns.yml'
defs = load_patterns(path)

len(unique_function_names)

67

In [39]:

for d in unique_function_names:
	ctx = Controller(ClauseGenerator(d))
	out = ctx['statement']
	# print(out)



A is a point such that ∠LMN equals ∠NMA.
L is the center of the circle that passes through N, P, and K.
C is the circumcenter of triangle HNY.
H, O, C, and K forms a quadrilateral where line KC equals line HO.
Z, O, F, and R forms a trapezoid where line ZO is congruent to line FR.
W, N, V is an equilateral triangle.
D is a point such that ∠RND is congruent to ∠DIR.
Line JO, line EB are congruent in quadrilateral EJBO.
D is a point where line FV equals line DC.
N is the foot of J on line QO.
A is a point.
G is located at the concurrence of the angle bisectors of triangle EYL.
L is located at the incenter of U, W, and I with touchpoints H, C, & E.
B is the excenter of triangle LSH.
Y is at the excenter of triangle SUJ with touchpoints A, B, and I.
K is the point of intersection of the medians of triangle PXU.
Triangle KVT has nine-point center E and the circle intersects the sides line VT, line TK, and line KV at A, B, and D respectively.
Circle centered at S with radius SN meets circle

In [42]:
ctx['formal']

'on_circum K L Z X'